# LSOA catchment areas for stroke units

Usually we assume that when an ambulance is taking a patient to an acute stroke unit, the ambulance will choose to go to the nearest acute stroke unit out of all available options.

Every area of the UK is in a single Lower Super Output Area (LSOA). By calculating the travel time from every LSOA to every stroke unit, we can calculate which acute stroke unit is closest to a patient in any LSOA.

This notebook shows how to use the `stroke-maps` package to find the catchment area of each acute stroke unit. It does this by checking each LSOA in turn to find out which stroke unit is nearest to it.

## Notebook setup

In [1]:
import stroke_maps.load_data
import stroke_maps.units  # for transfer units
from stroke_maps.catchment import find_each_lsoa_chosen_unit

import pandas as pd

## Load data

__LSOA travel time__

This dataframe contains one row for each LSOA in England and Wales. There is one column for each stroke unit in England and Wales. The value in each cell is the time from that row's LSOA to that column's stroke unit.

In [2]:
df_travel_lsoa = stroke_maps.load_data.travel_time_matrix_lsoa()

# Show the first five rows and columns:
df_travel_lsoa.iloc[:5, :5]

,B152TH,B714HJ,B95SS,BA13NG,BA214AT
LSOA,,,,,
Adur 001A,173.3,179.8,171.2,161.5,152.9
Adur 001B,173.3,179.8,172.3,161.5,152.9
Adur 001C,173.3,180.9,172.3,150.8,151.9
Adur 001D,173.3,180.9,172.3,161.5,152.9
Adur 001E,174.4,180.9,173.3,150.8,151.9


Write how many LSOA and then how many stroke teams are in this matrix:

In [3]:
df_travel_lsoa.shape

(34752, 125)

__Stroke unit services__

This dataframe contains information on which stroke units provide which services (IVT and MT). The imported dataframe, df_units, contains only units that appear in the travel time matrix. This includes units that are not acute stroke units (i.e. provide neither thrombolysis nor thrombectomy).

In [4]:
df_units = stroke_maps.load_data.stroke_unit_region_lookup()

df_units.head(3).T

postcode,SY231ER,CB20QQ,L97AL
stroke_team,Bronglais Hospital (Aberystwyth),"Addenbrooke's Hospital, Cambridge","University Hospital Aintree, Liverpool"
short_code,AB,AD,AI
ssnap_name,Bronglais Hospital,Addenbrooke's Hospital,University Hospital Aintree
use_ivt,1,1,1
use_mt,0,1,1
use_msu,0,1,1
transfer_unit_postcode,nearest,nearest,nearest
lsoa,Ceredigion 002A,Cambridge 013D,Liverpool 005A
lsoa_code,W01000512,E01017995,E01006654
region,Hywel Dda University Health Board,NHS Cambridgeshire and Peterborough ICB - 06H,NHS Cheshire and Merseyside ICB - 99A


## Calculate nearest acute stroke unit

The full travel time matrix gives the time from every LSOA to every stroke unit, including those that don't provide acute care. Realistically an ambulance should never choose to take the patient to those units.

Firstly, find a list of stroke units that offer acute care. These are units where `use_ivt` and/or `use_mt` are equal to 1 (one).

In [5]:
mask_acute = ((df_units['use_ivt'] == 1) | (df_units['use_mt'] == 1))

postcodes_acute_units = df_units[mask_acute].index.values

Check how many postcodes are in this list compared with how many stroke units are in the full dataframe:

In [20]:
n_units = len(df_units)
n_units_acute = len(postcodes_acute_units)

print(n_units, n_units_acute)

141 113


Now make a copy of the travel time matrix that only includes times to these units:

In [8]:
df_travel_lsoa_acute = df_travel_lsoa[postcodes_acute_units].copy()

# (Number of LSOA, number of stroke teams)
df_travel_lsoa_acute.shape

(34752, 113)

Run the function to find which of the units in the reduced dataframe is closest to each LSOA:

In [9]:
df_catchment_acute = find_each_lsoa_chosen_unit(df_travel_lsoa_acute)

df_catchment_acute.head(5)

,unit_travel_time,unit_postcode
LSOA,,
Adur 001A,17.6,BN25BE
Adur 001B,18.7,BN25BE
Adur 001C,17.6,BN112DH
Adur 001D,17.6,BN112DH
Adur 001E,16.5,BN112DH


## Calculate nearest thrombectomy unit

We can use a very similar method to before by only keeping stroke units that offer thrombectomy (MT):

In [10]:
mask_mt = (df_units['use_mt'] == 1)

postcodes_mt_units = df_units[mask_mt].index.values

Check how many postcodes are in this list compared with how many stroke units are in the full dataframe:

In [19]:
n_units = len(df_units)
n_units_mt = len(postcodes_mt_units)

print(n_units, n_units_mt)

141 25


Now make a copy of the travel time matrix that includes times to only these units:

In [13]:
df_travel_lsoa_mt = df_travel_lsoa[postcodes_mt_units].copy()

# (Number of LSOA, number of stroke teams)
df_travel_lsoa_mt.shape

(34752, 25)

Run the function to find which of the units in the reduced dataframe is closest to each LSOA:

In [14]:
df_catchment_mt = find_each_lsoa_chosen_unit(df_travel_lsoa_mt)

df_catchment_mt.head(5)

,unit_travel_time,unit_postcode
LSOA,,
Adur 001A,17.6,BN25BE
Adur 001B,18.7,BN25BE
Adur 001C,19.8,BN25BE
Adur 001D,19.8,BN25BE
Adur 001E,19.8,BN25BE


## Calculate nearest transfer unit

The method for finding transfer units is shown in more detail in _the transfer units demonstration_ (__TO DO__: add link to this).

In [15]:
df_transfer = stroke_maps.units.calculate_transfer_units(df_units)

In [16]:
df_transfer

,transfer_unit_travel_time,transfer_unit_postcode
postcode,,
SY231ER,135.8,CF144XW
CB20QQ,0.0,CB20QQ
L97AL,0.0,L97AL
CH495PE,27.3,L97AL
BA13NG,33.7,BS105NB
...,...,...
SL24HL,NaN,<NA>
HP112TT,34.8,OX39DU
BA214AT,74.5,BS105NB


## Combine results for nearest acute, MT, and transfer units

In [17]:
# Combine nearest unit and nearest MT unit:
df_results = pd.merge(
    df_catchment_acute, df_catchment_mt,
    left_index=True, right_index=True, how='left',
    suffixes=['_acute', '_mt']
)
# Combine these with the nearest unit's transfer unit:
df_results = pd.merge(
    df_results, df_transfer,
    left_on='unit_postcode_acute', right_index=True, how='left',
)

In [18]:
df_results.head(5)

,unit_travel_time_acute,unit_postcode_acute,unit_travel_time_mt,unit_postcode_mt,transfer_unit_travel_time,transfer_unit_postcode
LSOA,,,,,,
Adur 001A,17.6,BN25BE,17.6,BN25BE,0.0,BN25BE
Adur 001B,18.7,BN25BE,18.7,BN25BE,0.0,BN25BE
Adur 001C,17.6,BN112DH,19.8,BN25BE,31.6,BN25BE
Adur 001D,17.6,BN112DH,19.8,BN25BE,31.6,BN25BE
Adur 001E,16.5,BN112DH,19.8,BN25BE,31.6,BN25BE


## Example: turning Birmingham into an island

Ordinarily the catchment areas of each stroke unit don't match up nicely to other defined regions, for example the Integrated Care Boards or Ambulance Service boundaries.

Sometimes though it's preferable to only consider patients within a given Integrated Care Board region, even if patients outside the border would still ordinarily travel to a stroke unit within that ICB.

This is quite like pretending that everything outside the ICB doesn't exist. Outside the ICB, there are no LSOA with patients who will want to enter the ICB for treatment. Outside the ICB, there are no other stroke units that patients within the ICB might prefer to travel to.

When the regions are split off like this, it is as though each one is its own island with nothing else surrounding it.

To create this effect with the catchment code, it is possible to restrict:
+ which stroke units are considered for each LSOA
+ which LSOA are considered